# Importing **Libraries** 
---


In [102]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow import keras

# Reading Data

---


In [79]:
data = pd.read_csv("hotel_bookings.csv").fillna(0)
data.head(10)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,0.0,0.0,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,0.0,0.0,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,0.0,0.0,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,0.0,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,0.0,0,Transient,98.0,0,1,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,0.0,0,Transient,98.0,0,1,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015,July,27,1,0,2,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,0.0,0.0,0,Transient,107.0,0,0,Check-Out,2015-07-03
7,Resort Hotel,0,9,2015,July,27,1,0,2,2,0.0,0,FB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,303.0,0.0,0,Transient,103.0,0,1,Check-Out,2015-07-03
8,Resort Hotel,1,85,2015,July,27,1,0,3,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,0.0,0,Transient,82.0,0,1,Canceled,2015-05-06
9,Resort Hotel,1,75,2015,July,27,1,0,3,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,15.0,0.0,0,Transient,105.5,0,0,Canceled,2015-04-22


#data preprocess
---

In [80]:
le = preprocessing.LabelEncoder()
data["arrival_date_month"] = le.fit_transform(data["arrival_date_month"])
data["hotel"] = le.fit_transform(data["hotel"])
data["meal"] = le.fit_transform(data["meal"])
data["country"] = le.fit_transform(data["country"].astype(str))
data["market_segment"] = le.fit_transform(data["market_segment"])
data["distribution_channel"] = le.fit_transform(data["distribution_channel"])
data["reserved_room_type"] = le.fit_transform(data["reserved_room_type"])
data["assigned_room_type"] = le.fit_transform(data["assigned_room_type"])
data["deposit_type"] = le.fit_transform(data["deposit_type"])
data["customer_type"] = le.fit_transform(data["customer_type"])
data["reservation_status"] = le.fit_transform(data["reservation_status"])
data['reservation_status_date'] = data[['reservation_status_year', 'reservation_status_month', 'reservation_status_day']] = data['reservation_status_date'].str.split("-",expand=True) 
data = data.drop(columns='reservation_status_date', axis=1)

data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_year,reservation_status_month,reservation_status_day
0,1,0,342,2015,5,27,1,0,0,2,0.0,0,0,136,3,1,0,0,0,2,2,3,0,0.0,0.0,0,2,0.0,0,0,1,2015,07,01
1,1,0,737,2015,5,27,1,0,0,2,0.0,0,0,136,3,1,0,0,0,2,2,4,0,0.0,0.0,0,2,0.0,0,0,1,2015,07,01
2,1,0,7,2015,5,27,1,0,1,1,0.0,0,0,60,3,1,0,0,0,0,2,0,0,0.0,0.0,0,2,75.0,0,0,1,2015,07,02
3,1,0,13,2015,5,27,1,0,1,1,0.0,0,0,60,2,0,0,0,0,0,0,0,0,304.0,0.0,0,2,75.0,0,0,1,2015,07,02
4,1,0,14,2015,5,27,1,0,2,2,0.0,0,0,60,6,3,0,0,0,0,0,0,0,240.0,0.0,0,2,98.0,0,1,1,2015,07,03


# Train and Test Split

---


In [86]:
y = data['is_canceled']
x = pd.DataFrame(data.drop(['is_canceled'], axis=1))
xFeatures = x.columns.values 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, shuffle = True)

x_train

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_year,reservation_status_month,reservation_status_day
32433,1,0,2017,6,25,20,0,3,1,0.0,0,0,136,2,0,1,0,4,0,3,0,0,0.0,254.0,0,2,121.5,0,0,1,2017,06,23
52826,0,69,2016,6,25,12,2,4,1,0.0,0,0,136,5,3,0,0,0,0,0,0,1,236.0,0.0,0,2,120.0,0,0,0,2016,04,27
30649,1,21,2016,9,49,28,1,1,2,0.0,0,0,134,4,3,0,0,0,4,4,0,0,0.0,0.0,0,2,45.0,0,0,1,2016,11,30
14521,1,0,2016,4,4,20,0,1,2,0.0,0,0,0,3,1,0,0,1,4,4,0,0,0.0,0.0,0,2,61.0,0,1,1,2016,01,21
87436,0,0,2016,0,17,19,0,1,2,0.0,0,0,170,3,1,0,0,0,0,0,0,0,0.0,0.0,0,2,109.0,0,0,1,2016,04,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29853,1,28,2016,9,45,2,1,4,1,0.0,0,0,136,6,3,0,0,0,0,0,0,0,242.0,0.0,0,2,33.3,0,1,1,2016,11,07
936,1,49,2015,1,32,7,0,2,2,2.0,0,0,136,6,3,0,0,0,7,7,0,0,240.0,0.0,0,2,267.0,0,0,0,2015,06,22
36452,1,68,2017,8,20,20,0,1,2,0.0,0,0,136,6,3,0,0,0,4,8,2,0,240.0,0.0,0,2,127.0,1,2,1,2017,05,21
52945,0,71,2016,6,25,14,0,3,1,0.0,0,0,136,5,3,0,0,0,0,0,0,1,236.0,0.0,0,2,120.0,0,0,0,2016,04,27


# Training and Classification (Random Forest Method)

*Gini Criterion*

---

In [91]:
clf = RandomForestClassifier(criterion='gini',max_depth=2)
clf = clf.fit(x_train, y_train)

In [98]:
yPredict = clf.predict(x_test)
print(classification_report(y_test, yPredict))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     15057
           1       1.00      0.97      0.99      8821

    accuracy                           0.99     23878
   macro avg       0.99      0.99      0.99     23878
weighted avg       0.99      0.99      0.99     23878



*Entropy Criterion*


---




In [100]:
clf = RandomForestClassifier(criterion='entropy',max_depth=2)
clf = clf.fit(x_train, y_train)

In [101]:
yPredict = clf.predict(x_test)
print(classification_report(y_test, yPredict))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99     15057
           1       1.00      0.95      0.98      8821

    accuracy                           0.98     23878
   macro avg       0.99      0.98      0.98     23878
weighted avg       0.98      0.98      0.98     23878



# Training and Classification (Neural Network)


In [143]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(33, 1)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10),
    keras.layers.Dense(2)
])

In [144]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [145]:
# tst = np.random.randint(2, size=(95512, 1))
tst = np.zeros((95512, 1))
# model.fit(x_train, y_train, epochs=10)
# y_train = np.reshape(y_train, (95512, 1)) 
tst = np.asarray(tst)
y_train.shape


(95512,)

In [147]:
x_train = np.asarray(x_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

model.fit(x_train, y_train, epochs=10)


Epoch 1/10
2985/2985 [==============================] - 5s 2ms/step - loss: 1.2952 - accuracy: 0.8398
Epoch 2/10
2985/2985 [==============================] - 5s 2ms/step - loss: 0.4051 - accuracy: 0.9225
Epoch 3/10
2985/2985 [==============================] - 5s 2ms/step - loss: 0.2937 - accuracy: 0.9521
Epoch 4/10
2985/2985 [==============================] - 6s 2ms/step - loss: 0.2844 - accuracy: 0.9516
Epoch 5/10
2985/2985 [==============================] - 5s 2ms/step - loss: 0.1941 - accuracy: 0.9714
Epoch 6/10
2985/2985 [==============================] - 5s 2ms/step - loss: 0.1804 - accuracy: 0.9736
Epoch 7/10
2985/2985 [==============================] - 5s 2ms/step - loss: 0.1536 - accuracy: 0.9818
Epoch 8/10
2985/2985 [==============================] - 5s 2ms/step - loss: 0.1523 - accuracy: 0.9799
Epoch 9/10
2985/2985 [==============================] - 5s 2ms/step - loss: 0.1398 - accuracy: 0.9843
Epoch 10/10
2985/2985 [==============================] - 5s 2ms/step - loss: 0.130

In [152]:
x_test = np.asarray(x_test).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

747/747 - 1s - loss: 0.1275 - accuracy: 0.9892

Test accuracy: 0.9892369508743286


*Make Prediction*

---

In [164]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(x_test)

In [178]:
pred = np.zeros((len(predictions), 1))
for i in range(len(predictions)):
  pred[i] = np.argmax(predictions[i])

In [180]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     15057
         1.0       1.00      0.97      0.99      8821

    accuracy                           0.99     23878
   macro avg       0.99      0.99      0.99     23878
weighted avg       0.99      0.99      0.99     23878

